# Virtual Environment Configuration

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
!conda create -p env python=3.8 -y

In [ ]:
!source activate ./env; pip install bentoml --quiet
# !source activate ./env; pip install bentoml -U --quiet 

In [ ]:
!source activate ./env; pip install -U scikit-learn scipy matplotlib --quiet

# BentoML implementation on Colab

In [ ]:
%%writefile requirements.txt

bentoml==1.0.25
scikit-learn
cattrs==23.1.1

In [ ]:
!source activate ./env; pip install -r requirements.txt

In [ ]:
%%writefile bento_train.py

import bentoml

from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()
iris = load_iris()
X = iris.data[:, :4]
Y = iris.target
model.fit(X, Y)

bento_model = bentoml.sklearn.save_model('kneighbors', model)
print(f"Model saved: {bento_model}")

In [ ]:
!source activate ./env; python bento_train.py

In [ ]:
%%writefile bento_cmd.txt

bentoml==1.0.25
scikit-learn
cattrs==23.1.1

In [ ]:
%%writefile bento_test.py 

import bentoml

clf = bentoml.sklearn.get('kneighbors:latest').to_runner()
clf.init_local()
print(clf.predict.run([[2,3,4,5]]))

In [ ]:
!source activate ./env; python bento_test.py

In [ ]:
%%writefile service.py

import bentoml
import numpy as np
from bentoml.io import NumpyNdarry

clg = bentoml.sklearn.get('kneighbors:latest').to_runner()

service = bentoml.Service("kneighbors", runners=[clf])

# Create an API function
@service.api(input=NumpyNdarray(), output=NumpyNdarray())
def predict(input_series: np.ndarray) -> np.ndaray:

  result = clf.predict.run(input_series)

  return result

In [ ]:
!source activate ./env; bentoml serve service.py:service --reload

In [ ]:
%%writefile bentofile.yaml

service: "service.py:service"  # Same as the argument passed to `bentoml serve`
labels:
    owner: bentoml-team
    project: gallery
include:
- "*.py"  # A pattern for matching which files to include in the bento
python:
    packages:  # Additional pip packages required by the service
    - scikit-learn
    - pandas

In [ ]:
!source activate ./env; bentoml build